In [1]:
# ich dachte ich kopiere hier einfach schonmal den Code von dem Python Intro rüber, dann gehts schneller, wenn wir hier wirklich anfangen dran zu arbeiten
%load_ext autoreload
%autoreload 2
import numpy as np
#... startet bei Iteration 103

In [2]:
data = np.random.randint(0,101,(100,2))
centers = np.random.randint(0,101,(4,2))

In [3]:
clusters = np.expand_dims(centers, axis=1)
data = np.expand_dims(data, axis=0)

eucl = np.linalg.norm(clusters - data, axis=2)

labels = np.argmin(eucl, axis = 0)

In [4]:
def calc_distance(data,centers):
    clusters = np.expand_dims(centers, axis=1)
    data = np.expand_dims(data, axis=0)

    eucl = np.linalg.norm(clusters - data, axis=2)

    labels = np.argmin(eucl, axis = 0)
    return labels

In [5]:
calc_distance(data, 3)

AxisError: axis 1 is out of bounds for array of dimension 1

In [13]:
# k-means mit scikit learn library
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import data_exploration as de
import data_cleanup as dc
from time import time
import functions as fun

In [7]:
# Cleanup
data = pd.read_csv('../DMS_data/P53_HUMAN_Giacomelli_WT_Nutlin_2018.csv')
dat1 = dc.aufteilung_mut_pos(dc.min_max_norm(dc.z_transform(data)))
# Distance Matrix
dat1 = dc.rmv_na(de.mean_substitutions(dat1))
distance_mat: pd.DataFrame = de.dms_distance_matrix(dat1)
distance_mat

AS_old,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
AS_old,,,,,,,,,,,,,,,,,,,,
A,0.000000,1.845023,0.377937,0.422263,0.484488,0.453102,0.885071,1.315256,0.450027,0.319937,0.647783,0.579572,0.376269,0.303319,0.615500,0.382538,0.532855,1.383635,0.474408,1.654408
C,1.845023,0.000000,1.739546,1.693797,1.568963,1.592610,1.195729,0.893504,2.124593,1.716395,1.332638,1.456023,1.723635,1.877115,1.411988,1.696741,1.529327,0.776950,1.679513,0.862319
D,0.377937,1.739546,0.000000,0.355713,0.436921,0.451473,0.787570,1.179766,0.478785,0.292915,0.587418,0.604502,0.370503,0.396478,0.585682,0.333661,0.418055,1.256460,0.420888,1.565089
E,0.422263,1.693797,0.355713,0.000000,0.411267,0.418142,0.699191,1.144296,0.631467,0.316790,0.551906,0.503716,0.360214,0.427622,0.538216,0.277137,0.355378,1.200096,0.402335,1.474623
F,0.484488,1.568963,0.436921,0.411267,0.000000,0.360663,0.557544,1.053084,0.725772,0.425806,0.483683,0.490412,0.384107,0.530136,0.444300,0.430056,0.385896,1.095778,0.308844,1.303579
G,0.453102,1.592610,0.451473,0.418142,0.360663,0.000000,0.659749,1.109575,0.688432,0.402772,0.511603,0.458746,0.404120,0.410635,0.519944,0.370723,0.379697,1.143112,0.489493,1.403907
H,0.885071,1.195729,0.787570,0.699191,0.557544,0.659749,0.000000,0.756021,1.138538,0.772620,0.492955,0.602867,0.752086,0.932305,0.591549,0.743994,0.604865,0.816655,0.681989,0.986374
I,1.315256,0.893504,1.179766,1.144296,1.053084,1.109575,0.756021,0.000000,1.560230,1.125003,0.826347,1.008605,1.185265,1.345954,0.898532,1.170509,1.012057,0.527745,1.109897,0.819301
K,0.450027,2.124593,0.478785,0.631467,0.725772,0.688432,1.138538,1.560230,0.000000,0.541761,0.949036,0.928907,0.554754,0.464411,0.937452,0.554908,0.743065,1.612262,0.649373,1.912586


In [15]:
def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(
            data,
            estimator[-1].labels_,
            metric="euclidean",
            sample_size=300,
        )
    ]

    # Show the results
    formatter_result = (
        "{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}"
    )
    print(formatter_result.format(*results))

In [49]:
kmeans = KMeans(n_clusters=8, n_init='auto').fit(distance_mat)
# pca -> scatter plott

KMeans(n_init='auto')

In [51]:
kmeans.labels_

array([7, 4, 0, 0, 5, 5, 2, 3, 7, 0, 2, 6, 0, 7, 6, 0, 5, 3, 5, 1])

In [54]:
kmeans.feature_names_in_

array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P',
       'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], dtype=object)